In [ ]:
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)

import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
plt.rcParams["figure.figsize"] = (12,12)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
# GCS_PATH_ORI = KaggleDatasets().get_gcs_path('cassavafullrestfrecords')
# # GCS_PATH_NEW = KaggleDatasets().get_gcs_path('cldc-ds-1121-size-512')
# GCS_PATH_NEW = KaggleDatasets().get_gcs_path('cassavafullrestfrecords')
BATCH_SIZE = 16
IMAGE_SIZE = [600, 800]
# IMAGE_SIZE = [512, 512]
CLASSES = ['0', '1', '2', '3', '4']
NUM_CLASSES = len(CLASSES)
EPOCHS = 20

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.int32)# / 255.0
    return image

def read_tfrecord(example, labeled=True):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    }

    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    image = decode_image(example["image"])
    image_name = tf.cast(example["image_name"], tf.string)
    if labeled:
        label = tf.cast(example["target"], tf.int32)
        return image, label, image_name
    return image

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

def input_preprocess(image, label, image_name):
    image = tf.image.resize(image, size=IMAGE_SIZE)
    image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=255)
    image = tf.dtypes.cast(image, tf.int32)
    # normalize according to imagenet mean and standard deviation
#     image /= 255.0
#     image = (image - IMAGENET_MEAN) / IMAGENET_STD
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label, image_name

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [ ]:
def get_validation_dataset(FILENAMES, BATCH_SIZE=12):
    dataset = load_dataset(FILENAMES, labeled=True)
    dataset = dataset.map(input_preprocess, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [ ]:
TEST_FILENAMES = ['../MobileNet/full_res_TFRecords/Id_train18-1069.tfrec', 
                  '../MobileNet/full_res_TFRecords/Id_train19-1069.tfrec']

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
CLASSES = ['0', '1', '2', '3', '4']
NUM_CLASSES = len(CLASSES)

BATCH_SIZE = 36
train_dataset = get_validation_dataset(TEST_FILENAMES, BATCH_SIZE=BATCH_SIZE)

In [ ]:
for image, label, filename in train_dataset.take(1):
    pass

In [ ]:
image.shape

In [ ]:
plt.imshow(image[0])

In [ ]:
from keras.backend import sigmoid

class SwishActivation(tf.keras.layers.Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))



from keras.utils.generic_utils import get_custom_objects
# from tf.keras.layers import Activation
get_custom_objects().update({'swish_act': SwishActivation(swish_act)})

# from tensorflow.keras.experimental import CosineDecay
import efficientnet.keras as eff

# decay_steps = int(round(NUM_TRAINING_IMAGES/BATCH_SIZE))*EPOCHS
# cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)


inputs = tf.keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
#     x = data_augmentation_layers(inputs)
#     model = tf.keras.applications.EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")
model = eff.EfficientNetB0(include_top=False, input_tensor=inputs, weights=None)


# Freeze the pretrained weights
# model.trainable = False

# Rebuild top
x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = tf.keras.layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = tf.keras.layers.Dropout(top_dropout_rate, name="top_dropout")(x)

x = tf.keras.layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(swish_act)(x)
x = tf.keras.layers.Dropout(top_dropout_rate)(x)

x = tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(swish_act)(x)
x = tf.keras.layers.Dropout(top_dropout_rate)(x)

outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

# Compile
model = tf.keras.Model(inputs, outputs, name="EfficientNet")

loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.3)
#     loss = tf.keras.losses.CategoricalCrossentropy()

#     model.compile(loss=taylor_cross_entropy_loss(), optimizer=tf.keras.optimizers.Adam(cosine_decay), 
#               metrics=["accuracy"])

# model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(cosine_decay), 
#               metrics=["accuracy"])

In [ ]:
model = tf.keras.models.load_model('EfficientNet_B0_88243.h5')
# model_pseudolabel = tf.keras.models.load_model('EfficientNetb0_pseudolabel_train.h5', 
#                                   custom_objects = {"SwishActivation": SwishActivation})

# model = tf.keras.models.load_model('EfficientNetB0_512_512_87919.h5')
# model_pseudolabel = tf.keras.models.load_model('EfficientNetb0_512x512_pseudolabel_train.h5', 
#                                   custom_objects = {"SwishActivation": SwishActivation})

# model = tf.keras.models.load_model('EfficientNetB4_600_800_88739.h5')

In [ ]:
model.summary()

In [ ]:
NUM_VALIDATION_IMAGES = count_data_items(TEST_FILENAMES)

In [ ]:
NUM_VALIDATION_IMAGES

In [ ]:
labels = []
preds = []
filenames = []

i = 1

for image, label, filename in train_dataset.take(NUM_VALIDATION_IMAGES//BATCH_SIZE + 1):
    labels+=label.numpy().tolist()
    pred = 0.5*model.predict(image)
#     pred += 0.5*model_pseudolabel.predict(image)
#     pred = model.predict(image)
    preds+=pred.tolist()
    filenames+=filename.numpy().tolist()
    del image, label, filename
    print(i)
    i+=1

In [ ]:
len(filenames)

In [ ]:
y_label=[np.argmax(y) for y in labels]
y_pred=[np.argmax(y) for y in preds]
acc = accuracy_score(y_label, y_pred)
print(acc)

In [ ]:
plt.hist([np.max(y) for y in preds], bins=50)

In [ ]:
for i in range(len(y_label)):
    if y_label[i] != y_pred[i]:
        print('label: {0}, pred: {1}'.format(y_label[i], y_pred[i]))
        print(i, filenames[i])
        print('---------')

In [ ]:
df = pd.read_csv('../input/train.csv')
df['image_id'] = '../input/train_images/' + df['image_id']
df['label'] = df['label'].astype('string')

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=90,
                                                          width_shift_range=10,
                                                          height_shift_range=10,
                                                          horizontal_flip=True,
                                                          vertical_flip=True,) 
#                                                          dtype = 'uint8')

datagen = tf.keras.preprocessing.image.ImageDataGenerator()

train_datagen_flow = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,
    x_col='image_id',
    y_col='label',
    target_size=(600, 800),
    batch_size=32,
    subset='training',)
#     seed=12345)

In [ ]:
# model.evaluate(train_datagen_flow)

In [ ]:
image, label = train_datagen_flow.next()

In [ ]:
np.max(image)

In [ ]:
filenames = []
i = 1
for image, label in train_datagen_flow:
    filenames+=train_datagen_flow.filenames[i*32:(i+1)*32]
    print(filenames[0])
    print(i*32)
    i+=1
    plt.imshow(tf.dtypes.cast(image[0], tf.int32))
    plt.show()
    if i>0:break

In [ ]:
filenames[0]

In [ ]:
print(len(filenames))
print(len(set(filenames)))

In [ ]:
label[:16]

In [ ]:
plt.imshow(tf.dtypes.cast(image[0], tf.int32))

# Train on Pseudolabels

In [ ]:
import glob

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.int32)# / 255.0
    return image

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

def input_preprocess(image, label, image_name):
    image = tf.image.resize(image, size=IMAGE_SIZE)
    image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=255)
    image = tf.dtypes.cast(image, tf.int32)
    # normalize according to imagenet mean and standard deviation
#     image /= 255.0
#     image = (image - IMAGENET_MEAN) / IMAGENET_STD
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label, image_name

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

def read_tfrecord(example, labeled=True):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    }

    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
#     image = decode_image(example["image"])
    image_name = tf.cast(example["image_name"], tf.string)
    if labeled:
#         label = tf.cast(example["target"], tf.int32)
        return image_name
#     return image

def get_validation_dataset(FILENAMES, BATCH_SIZE=12):
    dataset = load_dataset(FILENAMES, labeled=True)
#     dataset = dataset.map(input_preprocess, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

# These files was the one the model was trained on

FILES = glob.glob('../MobileNet/full_res_TFRecords/*tfrec')

VALID = ['../MobileNet/full_res_TFRecords/Id_train11-1070.tfrec',
 '../MobileNet/full_res_TFRecords/Id_train05-1070.tfrec',
 '../MobileNet/full_res_TFRecords/Id_train02-1070.tfrec',
 '../MobileNet/full_res_TFRecords/Id_train01-1070.tfrec',
 '../MobileNet/full_res_TFRecords/Id_train13-1070.tfrec']

TEST = ['../MobileNet/full_res_TFRecords/Id_train18-1070.tfrec',
        '../MobileNet/full_res_TFRecords/Id_train19-1070.tfrec']

FILES = [f for f in FILES if f not in VALID and f not in TEST]

GET_FILENAMES = get_validation_dataset(FILES, BATCH_SIZE=120)

train_files = []

i = 1

for filename in GET_FILENAMES.take(count_data_items(FILES)//120 + 1):
    train_files+=(filename.numpy().tolist())
    tf.keras.backend.clear_session()
    print(i)
    i+=1
    
train_files = [t.decode() for t in train_files]

In [ ]:
train_files[0]

In [ ]:
len(train_files)

In [ ]:
conf_preds_file = []

model_pred = {}

for i in range(len(preds)):
    if np.max(preds[i]) > 0.65:
        conf_preds_file.append(filenames[i].decode())
        model_pred[filenames[i].decode()] = np.argmax(preds[i])

In [ ]:
len(conf_preds_file)

In [ ]:
model_pred

In [ ]:
df = pd.read_csv('../input/train.csv')

In [ ]:
df['image_id'] = '../input/train_images/' + df['image_id']

In [ ]:
# df = df[ (df['image_id'].isin(conf_preds_file)) | (df['image_id'].isin(train_files)) ]
df = df[ (df['image_id'].isin(conf_preds_file)) ]

In [ ]:
df.head()

In [ ]:
df['label'] = df.apply(lambda row: model_pred[row['image_id']] if row['image_id'] 
                                   in model_pred.keys() else row['label'], axis = 1)

In [ ]:
df['label'] = df['label'].astype('string')

In [ ]:
df.head()

In [ ]:
df.to_csv('restest.csv', index=False)

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator()
train_datagen_flow = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,
    x_col='image_id',
    y_col='label',
    target_size=(800, 600),
    batch_size=8,
    subset='training',
    seed=12345)

In [ ]:
model.fit(train_datagen_flow,
          steps_per_epoch=1356//8,
          epochs=10)